In [241]:
#All labraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
import json
import re
import shutil

In [242]:
# Training batch file

In [243]:
schema_path='C://Users//hp//7.Muffassil_Project//3_Wafer_fault_detection//3_Muffassil_code/schema_training.json'
training_folder_path='C:\\Users\\hp\\7.Muffassil_Project\\3_Wafer_fault_detection\\3_Muffassil_code\\Training_Batch_Files'

In [244]:
schema_path

'C://Users//hp//7.Muffassil_Project//3_Wafer_fault_detection//3_Muffassil_code/schema_training.json'

In [245]:
training_folder_path

'C:\\Users\\hp\\7.Muffassil_Project\\3_Wafer_fault_detection\\3_Muffassil_code\\Training_Batch_Files'

In [246]:
def valuesfromscheme():
    with open (schema_path, 'r') as f:
        data=json.load(f)   #conveting str to dict format ie we use json
#     print(type(data))
#     print(data)
    pattern=data['SampleFileName']
    LengthOfDateStampInFile=data['LengthOfDateStampInFile']
    LengthOfTimeStampInFile=data['LengthOfTimeStampInFile']
    NumberofColumns=data['NumberofColumns']
    
    return pattern,LengthOfDateStampInFile,LengthOfTimeStampInFile,NumberofColumns
    

In [247]:
pattern,LengthOfDateStampInFile,LengthOfTimeStampInFile,NumberofColumns=valuesfromscheme()

In [248]:
pattern

'wafer_08012020_120000.csv'

In [249]:
LengthOfDateStampInFile

8

In [250]:
LengthOfTimeStampInFile

6

In [251]:
NumberofColumns

592

In [252]:
def regex():
    regex="['wafer_']+[\d_]+[\d]+\.csv"
    return regex

In [253]:
regex=regex()

In [254]:
mkdir bad_data

In [255]:
mkdir good_data

In [256]:
def validationfilename (regex,LengthOfDateStampInFile,LengthOfTimeStampInFile):
    files=[i for i in os.listdir(training_folder_path)]
    for filename in files:
        if re.match(regex,filename):
            split_at_dot=re.split('.csv',filename)
            split_at_underline=re.split('_',split_at_dot[0])
            if len(split_at_underline[1])==LengthOfDateStampInFile:
                if len(split_at_underline[2])==LengthOfTimeStampInFile:
                    shutil.copy(f'{training_folder_path}\\{filename}','good_data')
                else:
                    shutil.copy(f'{training_folder_path}\\{filename}','bad_data')
                    
            else:
                    shutil.copy(f'{training_folder_path}\\{filename}','bad_data')
                    
        else:
                    shutil.copy(f'{training_folder_path}\\{filename}','bad_data')

In [257]:
validationfilename (regex,LengthOfDateStampInFile,LengthOfTimeStampInFile)

In [258]:
# validating column length in the files

def validate_column (NumberofColumns):
    files=[i for i in os.listdir('good_data') ]
    for filename in files:
        csv=pd.read_csv(f'good_data/{filename}')
        if csv.shape[1]==NumberofColumns:
            pass
        else: 
            shutil.move(f'good_data/{filename}','bad_data')
    

In [259]:
validate_column (NumberofColumns)

In [262]:
# Validate Missing Value in Whole Column

def Validate_missing_value_in_column():
    for file in os.listdir('good_data'):
        csv=pd.read_csv(f'good_data/{file}')
        count=0
        for column in csv:
            if csv[column].isnull().sum()==len(csv[column]):
#           if (len(csv[column])-csv[column].count())== len(csv[column]):
                count+=1
                shutil.move(f'good_data/{file}','bad_data')
                break
        if count==0:
            csv.rename(columns={'Unnamed: 0':'wafer'}, inplace=True)
            csv.to_csv(f'good_data/{file}', index=None, header=True)
    
    print('[info] validation of column done')
            
            
        
                                                         
                                                         

In [263]:
Validate_missing_value_in_column()

[info] validation of column done


In [273]:
# combine all the file into 1 file
list=[]
for file in os.listdir('good_data'):
    csv=pd.read_csv(f'good_data/{file}')
    list.append(csv)
    
combined=pd.concat(list,axis=0, ignore_index=True)
combined.to_csv('main_data.csv')



